In [ ]:
#|default_exp utils.__init__

In [ ]:
#|hide
from nbdev.showdoc import *

# Utility
> A collection of utility methods.

In [ ]:
#|export
import torch, random, gc
import numpy as np
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastcore.foundation import contextmanager
from fastai.callback.core import set_random_states, get_random_states

In [ ]:
#|export
def free_gpu_memory(learn:Learner, dls:DataLoaders=None):
    "Frees GPU memory using `gc.collect` and `torch.cuda.empty_cache`"
    learn.dls, learn, dls = None, None, None
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
#|export
@contextmanager
def less_random(seed=42, deterministic=True):
    "Stores and retrieves state of random number generators. Sets random seed for `random`, `torch`, and `numpy`. Does not set `torch.backends.cudnn.benchmark = False`"
    states = get_random_states()
    
    try: torch.manual_seed(seed)
    except NameError: pass
    try: torch.cuda.manual_seed_all(seed)
    except NameError: pass
    try: np.random.seed(seed%(2**32-1))
    except NameError: pass
    random.seed(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
    try:
        yield #we are managing global variables
    finally:
        set_random_states(**states)

A random state manager which provides some reproducibility without sacrificing potential training speed.

Unlike fast.ai's [`no_random`](https://docs.fast.ai/torch_core.html#no_random), `less_random` does not set `torch.backends.cudnn.benchmark = False` so it's possible to train faster. Training runs on the same GPU, PyTorch, & CUDA setup should be reproducible, but different hardware/software setup will probably have less reproducibility then using `no_random`.

In [ ]:
#|export

# modified from https://github.com/thomasbrandon/mish-cuda/blob/master/test/perftest.py
def scale_time(val:float, spec:str="#0.4G"):
    "Scale fractional second `time` values and return formatted to `spec`"
    if val == 0: return '-'
    PREFIXES = np.array([c for c in u"yzafpnµm kMGTPEZY"])
    exp = np.int8(np.log10(np.abs(val)) // 3 * 3 * np.sign(val))
    val /= 10.**exp
    prefix = PREFIXES[exp//3 + len(PREFIXES)//2]
    return f"{val:{spec}}{prefix}s"